## Exploring Ebay Car Sales Data

We will be working on a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. The version of the dataset we are working with is a sample of 50,000 data points that was prepared by Dataquest including simulating a less-cleaned version of the data.

The data dictionary provided with data is as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which which year the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which which year the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings. 
Let's start by importing the libraries we need and reading the dataset into pandas.

In [1]:
import pandas as pd
import numpy as np
import re

autos = pd.read_csv('autos.csv', encoding="ISO-8859-1")


In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

From the work we did, we can make the following observations:

- The dataset contains 20 columns, most of which are strings.
- Some columns have null values, but none have more than 20% null values.
- The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [3]:
def clear_names(str_):
    str_ = str_.replace('yearOfRegistration','registration_year')
    str_ = str_.replace('monthOfRegistration','registration_month')
    str_ = str_.replace('notRepairedDamage','unrepaired_damage')
    str_ = str_.replace('dateCreated','ad_created')
    str_ = re.sub(r'(?<!^)(?=[A-Z])', '_', str_).lower()
    return str_

new_column = []
for el in autos.columns:
    new_column.append(clear_names(el))

autos.columns = new_column
autos.columns


Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_p_s', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:
- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
- Examples of numeric data stored as text which can be cleaned and converted.

In [4]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 19:48:02,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Our initial observations:

There are a number of text columns where all (or nearly all) of the values are the same:
- seller
- offer_type
The num_photos column looks like the num_photos column has 0 for every column. We'll drop this column, plus the other two we noted as mostly one value.

In [5]:
autos = autos.drop(["nr_of_pictures", "seller", "offer_type"], axis=1)

The price and odometer columns are numeric values stored as text. For each column:

- Remove any non-numeric characters.
- Convert the column to a numeric dtype.

Use DataFrame.rename() to rename the column to odometer_km.

In [6]:
autos['price'] = (autos['price']
                  .str
                  .replace('$', '')
                  .str
                  .replace(',', '')
                  .astype(float)
                 )
autos['odometer'] = (autos['odometer']
                     .str
                     .replace('km', '')
                     .str
                      .replace(',', '')
                     .astype(float)
                    )
autos.rename({'odometer': 'odometer_km'},
                         axis=1, 
                         inplace=True
                        )
print(autos['price'].describe(include='all'))
print(autos['odometer_km'].describe(include='all'))


count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the <font color='red'>odometer_km</font> and <font color='red'>price</font> columns. Here's the steps we'll take:

- Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.

- We'll use:
    - <font color='red'>Series.unique().shape</font> to see how many unique values
    - <font color='red'>Series.describe()</font> to view min/max/median/mean etc
    - <font color='red'>Series.value_counts()</font>, with some variations:
        - chained to <font color='red'>.head()</font> if there are lots of values.
        - Because <font color='red'>Series.value_counts()</font> returns a series, we can use <font color='red'>Series.sort_index()</font> with <font color='red'>ascending= True</font> or <font color='red'>False</font> to view the highest and lowest values with their counts (can also chain to <font color='red'>head()</font> here).

In [7]:
print('odometer_km column:','\n',
      autos['odometer_km'].unique().shape, '\n','\n',
      autos['odometer_km'].describe(),'\n','\n',
      autos['odometer_km'].value_counts().sort_index(ascending= True).head(10),'\n','\n',
      autos['odometer_km'].value_counts().sort_index(ascending= False).head(10),'\n','\n',
      )

odometer_km column: 
 (13,) 
 
 count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64 
 
 5000.0      967
10000.0     264
20000.0     784
30000.0     789
40000.0     819
50000.0    1027
60000.0    1164
70000.0    1230
80000.0    1436
90000.0    1757
Name: odometer_km, dtype: int64 
 
 150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
40000.0       819
30000.0       789
Name: odometer_km, dtype: int64 
 



We can see that the values in 'odometer_km' field are rounded, which might indicate that sellers had to choose from pre-set options for this field. Additionally, there are more high mileage than low mileage vehicles.

In [8]:
print('price column:','\n',
      autos['price'].unique().shape, '\n','\n',
      autos['price'].describe(), '\n','\n',
      autos['price'].value_counts().sort_index(ascending= True).head(10), '\n','\n',
      autos['price'].value_counts().sort_index(ascending= False).head(10),'\n','\n',
      )

price column: 
 (2357,) 
 
 count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64 
 
 0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
Name: price, dtype: int64 
 
 99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
Name: price, dtype: int64 
 



Again, the prices in 'prices' column seem rounded, however given there are 2357 unique values in the column, that may just be people's tendency to round prices on the site.

There are a number of listings with prices below \$15, including about 1,500 at \$0. There are also a small number of listings with very high values, including 14 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. We will keep the \$1 items, but remove anything above \$350,000.

In [9]:
autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:

In [10]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


You'll notice that the first 10 characters represent the day (e.g. 2016-03-12). To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

In [11]:
autos['date_crawled'] = autos['date_crawled'].str[:10]
autos['ad_created'] = autos['ad_created'].str[:10]
autos['last_seen'] = autos['last_seen'].str[:10]

(autos['date_crawled']
        .value_counts(normalize=True, dropna=False)
        .sort_index()
)

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

It looks like the site was being crawled daily for about one month in March and April 2016. The distribution of lists scanned every day is about the same, with a slight increase on weekends.

In [12]:
(autos['ad_created']
        .value_counts(normalize=True, dropna=False)
        .sort_index()
)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

There is a large variety of ad created dates. Most fall within 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [13]:
(autos['last_seen']
        .value_counts(normalize=True, dropna=False)
        .sort_values()
)

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-18    0.007351
2016-03-08    0.007413
2016-03-13    0.008895
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-14    0.012602
2016-03-27    0.015649
2016-03-19    0.015834
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-26    0.016802
2016-03-23    0.018532
2016-03-25    0.019211
2016-03-24    0.019767
2016-03-21    0.020632
2016-03-20    0.020653
2016-03-28    0.020859
2016-03-22    0.021373
2016-03-29    0.022341
2016-04-01    0.022794
2016-03-31    0.023783
2016-03-12    0.023783
2016-04-04    0.024483
2016-03-30    0.024771
2016-04-02    0.024915
2016-04-03    0.025203
2016-03-17    0.028086
2016-04-05    0.124761
2016-04-07    0.131947
2016-04-06    0.221806
Name: last_seen, dtype: float64

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [14]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In general, the distribution looks quite normal, but there are some outliers that need to be analyzed and removed from the table.

In [15]:
(autos['registration_year']
        .value_counts(normalize=True, dropna=False)
        .sort_index()
)

1000    0.000021
1001    0.000021
1111    0.000021
1800    0.000041
1910    0.000103
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000041
1937    0.000082
1938    0.000021
1939    0.000021
1941    0.000041
1943    0.000021
1948    0.000021
1950    0.000062
1951    0.000041
1952    0.000021
1953    0.000021
1954    0.000041
1955    0.000041
1956    0.000082
1957    0.000041
1958    0.000082
1959    0.000124
1960    0.000474
1961    0.000124
1962    0.000082
1963    0.000165
1964    0.000247
          ...   
2000    0.064985
2001    0.054278
2002    0.051189
2003    0.055575
2004    0.055657
2005    0.060455
2006    0.054978
2007    0.046886
2008    0.045609
2009    0.042932
2010    0.032719
2011    0.033419
2012    0.026974
2013    0.016535
2014    0.013652
2015    0.008072
2016    0.025121
2017    0.028663
2018    0.009678
2019    0.000041
2800    0.000021
4100    0.000021
4500    0.000021
4800    0.000021
5000    0.000082
5911    0.000021
6200    0.000021
8888    0.0000

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [16]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
2008    0.047450
2009    0.044665
1997    0.041794
2011    0.034768
2010    0.034040
1996    0.029412
2012    0.028063
1995    0.026285
2016    0.026135
2013    0.017202
2014    0.014203
1994    0.013474
1993    0.009104
2015    0.008397
1992    0.007926
1990    0.007433
1991    0.007262
1989    0.003727
1988    0.002892
1985    0.002035
          ...   
1966    0.000471
1976    0.000450
1969    0.000407
1975    0.000386
1965    0.000364
1964    0.000257
1963    0.000171
1959    0.000129
1961    0.000129
1910    0.000107
1956    0.000086
1958    0.000086
1937    0.000086
1962    0.000086
1950    0.000064
1954    0.000043
1941    0.000043
1951    0.000043
1934    0.000043
1957    0.000043
1955    0.000043
1953    0.000021
1943    0.000021
1929    0.000021
1939    0.000021
1938    0.000021
1948    0.000021
1927    0.0000

It looks like most of the cars have been registered in the last 20 years.

 When working with data on cars, it's natural to explore variations across different car brands. We can use aggregation to understand the <font color='red'>brand</font> column. We'll explore the unique values in this column, and select the top 20 brands.

In [17]:
brands = autos["brand"].value_counts().head(10)
print(brands)

volkswagen       9862
bmw              5137
opel             5022
mercedes_benz    4503
audi             4041
ford             3263
renault          2201
peugeot          1393
fiat             1197
seat              853
Name: brand, dtype: int64


Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

Let's take a look at the average car price of the 10 most popular brands.

In [45]:
avr_price = {}
avr_odometer = {}
for el in brands.index:
    
    pr = autos.loc[autos['brand'] == el, 'price'].mean()
    odo = autos.loc[autos['brand'] == el, 'odometer_km'].mean()
    avr_price[el] = avr_price.get(el, 0) + int(pr)
    avr_odometer[el] = avr_odometer.get(el, 0) + int(odo)

    
bmp_series = pd.Series(avr_price)
df = pd.DataFrame(bmp_series, columns=['mean_price'])
m_m = pd.Series(avr_odometer)
df['mean_mileage'] = m_m
df

,mean_price,mean_mileage
audi,9336,129157
bmw,8332,132572
fiat,2813,117121
ford,3749,124266
mercedes_benz,8628,130788
opel,2975,129310
peugeot,3094,127153
renault,2474,128071
seat,4397,121131
volkswagen,5402,128707


Of the top 10 brands, there is a distinct price gap:

Audi, BMW and Mercedes Benz are more expensive
Ford, Opel, Peugeot, Fiat, Renault are less expensive
Volkswagen is in between - this may explain its popularity, it may be a 'best of both worlds' option. The range of car mileages does not vary as much as the prices do by brand.